In [2]:
import json
import openai
import gradio as gr
import re

In [43]:
def generate_story(prompt):
    client = openai.OpenAI(
        api_key = "ADD API KEY", 
    )
    
    messages = [{
        "role": "user",
        "content": prompt
    }]
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a creative writer of children's stories. "
                    "You write succinctly. You write stories that have clear scenes, "
                    "and vivid imagery that can pair easily with drawings/ illustrations."
                )
            },
            messages[0]
        ]
    )
    
    story = completion.choices[0].message.content
    print(story)
    return client, messages, story

In [14]:
sample_prompt = "a kid and a rbbit saves a forest"

client, messages, story = generate_story(sample_prompt)

**Title: Luna and the Brave Little Rabbit**

In a sunny village, deep in the heart of a lush green forest, lived a curious girl named Luna. She had bright blue eyes that sparkled like the morning dew and hair that danced in the wind. Each day after school, she would wander into the forest, exploring every nook and cranny.

One breezy afternoon, while picking wildflowers, Luna noticed something strange. The trees were whispering, their leaves trembling. A tiny rabbit hopped towards her, his fur as white as snow and his eyes big and round.

“Help! Help!” the rabbit squeaked with a quiver in his voice. “The forest is in danger!”

“Who are you?” Luna asked, kneeling down.

“I’m Benny, and I live in this forest. The river is drying up, and the trees are dying! We need to save our home!” He thumped his little paw on the ground.

Luna’s heart raced. “What can we do?”

Benny’s nose twitched. “If we can find the ancient Crystal of Life hidden in the Old Oak, it can bring back the water and the 

In [16]:
def generate_image_prompts(client, messages, story):
    messages.append({
        "role": "assistant",
        "content": [{ "type": "text", "text": story }]
    })

    messages.append({
        "role": "user",
        "content": [{ 
            "type": "text", 
            "text": (
                "Create separate image generation prompts for every scene. "
                "Describe the characters without using their names and add the "
                "character description to every prompt."
            )
        }]
    })

    print(json.dumps(messages, indent=4))
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    image_prompts = completion.choices[0].message.content
    print(image_prompts)
    return image_prompts

In [18]:
def generate_image_prompts(client, messages, story):
    messages.append({
        "role": "assistant",
        "content": story  # Adjusted to match the expected format
    })

    messages.append({
        "role": "user",
        "content": (
            "Create separate image generation prompts for every scene. "
            "Describe the characters without using their names and add the "
            "character description to every prompt."
        )
    })

    print(json.dumps(messages, indent=4))
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    image_prompts = completion.choices[0].message.content
    print(image_prompts)
    return image_prompts

In [22]:
image_prompts = generate_image_prompts(client, messages, story)

[
    {
        "role": "user",
        "content": "a kid and a rbbit saves a forest"
    },
    {
        "role": "assistant",
        "content": "**Title: Luna and the Brave Little Rabbit**\n\nIn a sunny village, deep in the heart of a lush green forest, lived a curious girl named Luna. She had bright blue eyes that sparkled like the morning dew and hair that danced in the wind. Each day after school, she would wander into the forest, exploring every nook and cranny.\n\nOne breezy afternoon, while picking wildflowers, Luna noticed something strange. The trees were whispering, their leaves trembling. A tiny rabbit hopped towards her, his fur as white as snow and his eyes big and round.\n\n\u201cHelp! Help!\u201d the rabbit squeaked with a quiver in his voice. \u201cThe forest is in danger!\u201d\n\n\u201cWho are you?\u201d Luna asked, kneeling down.\n\n\u201cI\u2019m Benny, and I live in this forest. The river is drying up, and the trees are dying! We need to save our home!\u201d He t

In [26]:
def process_image_prompts(image_prompts):
    pattern = r"Scene \d+:"
    prompts = re.split(pattern, image_prompts)
    prompts = [prompt.strip().replace("\n", " ") for prompt in prompts if prompt.strip()]
    for prompt in prompts:
        print(prompt)
    return prompts

In [28]:
sample_prompts = process_image_prompts(image_prompts)

Here are image generation prompts for each scene, including descriptions of the characters:  ---  ###
Luna Exploring the Forest **Prompt:** A curious young girl with bright blue eyes that sparkle like morning dew and flowing hair that dances in the wind. She is wearing a colorful dress adorned with wildflowers, surrounded by vibrant greenery in a sunny forest. Sunlight filters through the leaves above, casting playful shadows around her as she picks wildflowers.  ---  ###
Benny the Rabbit Approaches **Prompt:** A small rabbit with soft, snow-white fur and large, round eyes that convey urgency. He is standing on his hind legs, looking up at the girl with a worried expression in a sunlit glade. The scenery includes lush grass and a backdrop of tall trees, with the girl kneeling down to listen to him.  ---  ###
The Daring Journey Begins **Prompt:** The young girl and the little rabbit are running through a vibrant forest. The girl looks determined, her hair flowing behind her, while the r

In [30]:
def generate_images(client, prompts):
    image_urls = []
    for prompt in prompts:
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt + " Generate the image in the style of a children's book.",
            n=1,
            size="1024x1024"
        )
        image_url = response.data[0].url
        print(image_url)
        image_urls.append(image_url)
    return image_urls

In [34]:
sample_images = generate_images(client, sample_prompts)

https://oaidalleapiprodscus.blob.core.windows.net/private/org-q4y39tX3MUeRQdvqNWqrzLF8/user-z2oXLqnbMV7PyS2duEUH73Wf/img-PJXfltJGd7QtHq1j3HDhvkQt.png?st=2024-11-22T01%3A13%3A40Z&se=2024-11-22T03%3A13%3A40Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-11-22T00%3A10%3A14Z&ske=2024-11-23T00%3A10%3A14Z&sks=b&skv=2024-08-04&sig=EsFHcxeR7kk/2RVdSS0GQ9ejoWBHCYNRi2c%2BAxWKM64%3D
https://oaidalleapiprodscus.blob.core.windows.net/private/org-q4y39tX3MUeRQdvqNWqrzLF8/user-z2oXLqnbMV7PyS2duEUH73Wf/img-3M2MihfvpCcrTX2vIeuxxt4E.png?st=2024-11-22T01%3A13%3A50Z&se=2024-11-22T03%3A13%3A50Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-11-22T00%3A10%3A27Z&ske=2024-11-23T00%3A10%3A27Z&sks=b&skv=2024-08-04&sig=H%2BYzdup7Scprw5gwcnSgDV/s2/nNMSHM5C%2BzZdlO6sk%3D
https://oaidalleapiprodscus.blob.core.windows.ne

In [35]:
def main(prompt):
    client, messages, story = generate_story(prompt)
    image_prompts_text = generate_image_prompts(client, messages, story)
    prompts = process_image_prompts(image_prompts_text)
    image_urls = generate_images(client, prompts)
    return story, image_urls

In [41]:
iface = gr.Interface(
    fn=main,
    inputs=gr.Textbox(lines=5, placeholder="Enter your story prompt here..."),
    outputs=[
        gr.Textbox(label="Generated Story"),
        gr.Gallery(label="Generated Images")
    ],
    title="Children's Story Generator",
    description="Enter a prompt to generate a children's story and corresponding images."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
